# Cume Dist and Percent Rank

Currently Percent rank implemented by SQL Databases and Pandas Percent rank use different algorithms.

Pandas Percent Rank uses the same algoritm used by SQL Databases CumeDist function.

The calculation used by SQL Databases for CumeDist and Percent rank are:


$$
PERCENT\_RANK = (RANK – 1)/(COUNT -1)
$$

$$
CUME\_DIST = RANK/COUNT
$$


Current, [Ibis](http://ibis-project.org) uses the approach of Pandas, but probably should use the same 
behavior used by the SQL Databases.

More information about the difference between Percent Rank and CumeDist [here](https://www.sqlservercentral.com/articles/whats-the-difference-between-percent_rank-and-cume_dist)


Some Ibis reference about the current implementation:

- https://github.com/ibis-project/ibis/blob/1.3.0/ibis/pandas/execution/window.py#L360
- https://github.com/ibis-project/ibis/blob/1.3.0/ibis/tests/all/test_window.py#L41


This notebook aims to implement a function for sql percent_rank using pandas and test it against 
[OmniSciDB](https://docs.omnisci.com/).

In [46]:
from copy import copy

import ibis
import pandas as pd
# local
from settings import conf
from utils import cursor2df

### Setup

In [47]:
BACKENDS = ['omniscidb']
con = {
    backend: getattr(ibis, backend).connect(**conf[backend]) 
    for backend in BACKENDS
}

In [48]:
t = con['omniscidb'].table('functional_alltypes')

### Percent Rank and CumeDist definition

In [54]:
def win_count(se):
    count = copy(se)
    count[:] = len(count)
    return count


def sql_percent_rank(se):
    """
    PERCENT_RANK = (RANK – 1)/(COUNT -1)
    """
    return (se.rank(method='min') - 1) / (se.transform(len) - 1)


def sql_cume_dist(se):
    """
    CUME_DIST = RANK/COUNT
    """
    return se.rank(method='min') / se.transform(len)


def pd_percent_rank(se): 
    return se.rank(method='min', pct=True)

def pd_percent_rank2(se): 
    return se.rank(pct=True)

### Test CUME DIST using Ibis/OmniSciDB

In [55]:
alltypes = t.execute()

analytic_alltypes = alltypes.sort_values('id').groupby('string_col')

result = alltypes.assign(
    pandas_pct_rank=pd_percent_rank(analytic_alltypes.id),
    cume_dist=sql_cume_dist(analytic_alltypes.id),
    pct_rank=sql_percent_rank(analytic_alltypes.id),
).set_index('id').sort_index()

# display(result[['pandas_pct_rank', 'cume_dist']].describe())
pd.testing.assert_series_equal(
    result['pandas_pct_rank'], result['cume_dist'],
    check_dtype=False,
    check_names=False,
)

In [69]:

# (data.rank(method='min') - 1) / (len(data) - 1) and data.rank(method='min', pct=True)

col = alltypes.string_col

r1 = (col.rank(method='min') - 1) / (len(col) - 1)

r1 = col.rank(method='min', pct=True)
r2 = col.rank(method='min')  / len(col) 
pd.testing.assert_series_equal(r1, r2)

In [58]:
cume_dist = sql_cume_dist(analytic_alltypes.id)
pct_rank=sql_percent_rank(analytic_alltypes.id)
pandas_pct_rank=pd_percent_rank(analytic_alltypes.id)
pandas_pct_rank2=pd_percent_rank2(analytic_alltypes.id)

In [59]:
pd.testing.assert_series_equal(
    pandas_pct_rank, pandas_pct_rank2,
    check_dtype=False,
    check_names=False,
)

In [51]:
data = pd.DataFrame({'name': ['foo', 'bar', 'foo'],
                       'value': [1, 2, 3], 'index': [1,2,3]})

g = data.groupby('name')['value']
print(g.apply(win_count))
print(g.transform(len))

0    2
1    1
2    2
Name: value, dtype: int64
0    2
1    1
2    2
Name: value, dtype: int64


In [52]:
data = pd.DataFrame({'name': ['foo', 'bar', 'foo'],
                       'value': [1, 2, 3], 'index': [1,2,3]})

g = data.value
print(g.apply(win_count))
print(g.transform(len))

TypeError: object of type 'int' has no len()

### Test OmniSciDB Percent Rank

In [ ]:
sql = '''
SELECT 
    id, cume_dist() OVER (PARTITION BY "string_col" ORDER BY "id") AS cume_dist_val,
    percent_rank() OVER (PARTITION BY "string_col" ORDER BY "id") AS percent_rank_val
FROM functional_alltypes
'''

cur = con['omniscidb'].con.execute(sql)
df = cursor2df(cur).set_index('id').sort_index()
df.head()

In [ ]:
pd.testing.assert_series_equal(
    result['cume_dist'], 
    df['cume_dist_val'],
    check_dtype=False,
    check_names=False,
)

pd.testing.assert_series_equal(
    result['cume_dist'], 
    df['cume_dist_val'],
    check_dtype=False,
    check_names=False,
)

### MySQL

In [ ]:
con['mysql'] = ibis.mysql.connect(**conf['mysql'])

sql = '''
SELECT 
    t0.`index`, t0.`Unnamed: 0`, t0.id, t0.bool_col, 
    t0.tinyint_col, t0.smallint_col, t0.int_col, 
    t0.bigint_col, t0.float_col, t0.double_col, 
    t0.date_string_col, t0.string_col, 
    t0.timestamp_col, t0.year, t0.month, 
    percent_rank() OVER (PARTITION BY t0.string_col ORDER BY t0.id ASC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS val 
FROM (
    SELECT t1.`index` AS `index`, t1.`Unnamed: 0` AS `Unnamed: 0`, t1.id AS id, 
    t1.bool_col = 1 AS bool_col, t1.tinyint_col AS tinyint_col, t1.smallint_col AS smallint_col, 
    t1.int_col AS int_col, t1.bigint_col AS bigint_col, t1.float_col AS float_col, 
    t1.double_col AS double_col, t1.date_string_col AS date_string_col, t1.string_col AS string_col, 
    t1.timestamp_col AS timestamp_col, t1.year AS year, t1.month AS month 
FROM functional_alltypes AS t1) AS t0 
LIMIT 10000
'''

con['mysql'].con.execute(sql)

In [ ]:
sql = '''
SELECT 
    t0.`index`, t0.`Unnamed: 0`, t0.id, t0.bool_col, t0.tinyint_col,
    t0.smallint_col, t0.int_col, t0.bigint_col, t0.float_col, t0.double_col,
    t0.date_string_col, t0.string_col, t0.timestamp_col, t0.year, t0.month,
    percent_rank() OVER (PARTITION BY t0.string_col ORDER BY t0.id ASC) AS val
FROM (
    SELECT t1.`index` AS `index`, t1.`Unnamed: 0` AS `Unnamed: 0`, t1.id AS id, 
        t1.bool_col = 1 AS bool_col, 
        t1.tinyint_col AS tinyint_col, t1.smallint_col AS smallint_col, 
        t1.int_col AS int_col, t1.bigint_col AS bigint_col, t1.float_col AS float_col,
        t1.double_col AS double_col, t1.date_string_col AS date_string_col,
        t1.string_col AS string_col, t1.timestamp_col AS timestamp_col,
        t1.year AS year, t1.month AS month
    FROM functional_alltypes AS t1
) AS t0
'''
con['mysql'].con.execute(sql)

In [10]:
t.execute().head()

,index,Unnamed__0,id,bool_col,tinyint_col,smallint_col,int_col,bigint_col,float_col,double_col,date_string_col,string_col,timestamp_col,year_,month_
0,0,0,6690,True,0,0,0,0,0.0,0.0,11/01/10,0,2010-11-01 00:00:00,2010,11
1,1,1,6691,False,1,1,1,10,1.1,10.1,11/01/10,1,2010-11-01 00:01:00,2010,11
2,2,2,6692,True,2,2,2,20,2.2,20.2,11/01/10,2,2010-11-01 00:02:00,2010,11
3,3,3,6693,False,3,3,3,30,3.3,30.3,11/01/10,3,2010-11-01 00:03:00,2010,11
4,4,4,6694,True,4,4,4,40,4.4,40.4,11/01/10,4,2010-11-01 00:04:00,2010,11


In [15]:
result_fn = lambda t, win: t.id.percent_rank().over(win)
expected_fn = lambda t: (
    (t.id.rank(method='min') - 1) / (t.id.transform(len) - 1)
)

alltypes = t
df = alltypes.execute()

expr = alltypes.mutate(
    val=result_fn(
        alltypes,
        win=ibis.window(
            following=0,
            group_by=[alltypes.string_col],
            order_by=[alltypes.id],
        ),
    )
)

result = expr.execute().set_index('id').sort_index()
column = expected_fn(df.sort_values('id').groupby('string_col'))
expected = df.assign(val=column).set_index('id').sort_index()

left, right = result.val, expected.val

pd.testing.assert_series_equal(left, right)

In [19]:
left.max()

1.0

## Conclusions

This notebook implemented Percent Rank and CumeDist for pandas. The tests against OmniSciDB passed with success.  